In [ ]:
# ----------------------------------------------------------------------
# Grid Search (그리드 탐색) 정의 및 원리
# ----------------------------------------------------------------------

# 1. 정의:
# - 사용자가 지정한 '하이퍼파라미터'들의 모든 가능한 조합(격자, Grid)을 '완전 탐색'하여 최적의 성능을 내는 조합을 찾는 방식.
# - 지도 학습(Supervised Learning) 환경에서 모델의 일반화 성능을 높이는 데 주로 사용됨.

# 2. 핵심 원리:
# - 탐색 공간 정의: 튜닝할 각 하이퍼파라미터에 대해 후보 값들의 리스트를 지정.
# - 완전 탐색: 지정된 리스트의 '데카르트 곱(Cartesian Product)'에 해당하는 모든 조합을 하나씩 시도.
# - 교차 검증 (CV): 각 조합에 대해 K-Fold 교차 검증을 수행하여 일반화 성능(평균 점수)을 평가.
# - 최적 선택: 가장 높은 평균 교차 검증 점수를 기록한 하이퍼파라미터 조합을 최종 선택.

# 3. 장점:
# - 간결성: 구현이 매우 간단하고 직관적.
# - 확실성: 정의된 탐색 공간 내에서는 최적의 조합을 놓치지 않고 찾음.

# 4. 단점:
# - 비효율성: 탐색 공간(하이퍼파라미터 개수나 후보 값의 범위)이 넓어지면, 탐색 시간이 기하급수적으로 증가함.
# - 자원 소모: 계산 비용과 시간이 많이 소모되므로, 대규모 데이터셋이나 복잡한 모델에는 부적합할 수 있음.
# - Random Search나 Optuna 같은 지능형 탐색 기법에 비해 시간이 비효율적임.

# ----------------------------------------------------------------------
# Python에서의 구현 (Scikit-learn)
# ----------------------------------------------------------------------

# - 클래스: sklearn.model_selection.GridSearchCV
# - 주요 매개변수:
#   - estimator: 튜닝할 머신러닝 모델 객체.
#   - param_grid: 하이퍼파라미터 후보 값들을 담은 딕셔너리.
#   - cv: 사용할 교차 검증 폴드(Fold) 개수 (예: cv=5).
#   - scoring: 평가 지표 (예: 'accuracy', 'f1', 'neg_mean_squared_error').

# - 주요 결과 속성:
#   - .best_params_: 최적의 성능을 낸 하이퍼파라미터 조합 딕셔너리.
#   - .best_score_: 최적의 조합으로 얻은 최고 교차 검증 평균 점수.
#   - .best_estimator_: 최적의 하이퍼파라미터로 재학습된 최종 모델 객체.

In [ ]:
# --------------------
# 라이브러리 및 데이터 준비
# --------------------
import numpy as np
from sklearn.model_selection import GridSearchCV # Grid Search 함수
from sklearn.ensemble import RandomForestClassifier # 탐색 대상 모델
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Iris 데이터셋 로드
iris = load_iris()
X, y = iris.data, iris.target

# 데이터 분할 (Grid Search 내부에서 교차 검증을 수행하므로 분할이 필수는 아님)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("--- 데이터 준비 완료 ---")

In [ ]:
# --------------------
#  2단계: 탐색 공간 (Parameter Grid) 정의
# --------------------
# n_estimators: 트리의 개수 (100, 200)
# max_depth: 트리의 최대 깊이 (3, 5, 7)
# 총 시도 횟수 = 2 * 3 = 6가지 조합
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [10, 20] # leaf 노드가 되기 위한 최소 샘플 수
}

print("\n--- 2단계: 탐색할 하이퍼파라미터 조합 수 ---")
# n_estimators(2) * max_depth(3) * min_samples_leaf(2) = 12가지 조합
total_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"총 시도할 조합의 수: {total_combinations}가지")

In [ ]:
# --------------------
# 3단계: Grid Search 실행 및 최적 조합 찾기
# --------------------
# estimator: 사용할 모델 (랜덤 포레스트)
# param_grid: 정의된 탐색 공간
# cv=5: 5-Fold 교차 검증 사용
# scoring='accuracy': 평가 지표는 정확도(Accuracy) 사용
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    scoring='accuracy',
    n_jobs=-1 # 모든 코어 사용
)

# 학습 데이터에 Grid Search 실행
grid_search.fit(X_train, y_train)

# --------------------
# 4단계: 결과 확인
# --------------------
print("\n--- 4단계: 최적의 하이퍼파라미터 조합 및 성능 ---")
# 최적의 하이퍼파라미터 조합 (Best Parameters)
print("Best Parameters:", grid_search.best_params_)

# 해당 조합으로 얻은 최고 교차 검증 점수 (Best Score)
print(f"Best CV Score (Accuracy): {grid_search.best_score_:.4f}")

# 최종 모델: 최적 하이퍼파라미터로 재학습된 모델
best_model = grid_search.best_estimator_
final_test_score = best_model.score(X_test, y_test)
print(f"Final Test Score: {final_test_score:.4f}")